### Check to make sure we can access the space station satellite data.

In [1]:
import requests

def get_tle_data(url):
    response = requests.get(url)
    return response.text

def tle_lines_to_list(tle):
    split_data = tle.strip().split("\n")
    stripped_data = [line.strip() for line in split_data]
    return stripped_data

tle_lines = tle_lines_to_list(get_tle_data("https://celestrak.org/NORAD/elements/gp.php?GROUP=stations&FORMAT=tle"))
tle_lines


['ISS (ZARYA)',
 '1 25544U 98067A   25266.24686783  .00027783  00000+0  48333-3 0  9995',
 '2 25544  51.6338 181.0318 0004269   9.4814 350.6255 15.50560509530377',
 'CSS (TIANHE)',
 '1 48274U 21035A   25267.01613056  .00044487  00000+0  53348-3 0  9992',
 '2 48274  41.4637 141.3941 0001955  57.4402 302.6625 15.60118219251575',
 'ISS (NAUKA)',
 '1 49044U 21066A   25266.24686783  .00027783  00000+0  48333-3 0  9991',
 '2 49044  51.6338 181.0318 0004269   9.4814 350.6255 15.50560509230443',
 'FREGAT DEB',
 '1 49271U 11037PF  25266.62441859  .00015456  00000+0  27551-1 0  9996',
 '2 49271  51.6358  76.4026 0940601  49.6020 318.3159 12.33291795192047',
 'CSS (WENTIAN)',
 '1 53239U 22085A   25267.01613056  .00044487  00000+0  53348-3 0  9995',
 '2 53239  41.4637 141.3941 0001955  57.4402 302.6625 15.60118219173845',
 'CSS (MENGTIAN)',
 '1 54216U 22143A   25267.01613056  .00044487  00000+0  53348-3 0  9996',
 '2 54216  41.4637 141.3941 0001955  57.4402 302.6625 15.60118219158642',
 'SOYUZ-MS 

### Create nested dictionary of space station TLE data:

In [2]:
from datetime import datetime, timedelta


def tle_notation_to_float(s):
    s = s.strip()
    if not s or s == "00000+0":
        return 0.0
    base, exp = s[:-2], s[-2:]
    base = "0" + "." + base
    return float(f"{base}e{exp}")


def epoch_to_datetime(epoch):
    year = int(epoch[0:2])
    day_of_year = float(epoch[2:])
    year += 2000 if year < 57 else 1900

    epoch_base = datetime(year, 1, 1)

    epoch_datetime = epoch_base + timedelta(days=day_of_year - 1)

    return epoch_datetime


def datetime_to_jd(dt):
    jd = dt.toordinal() + 1721424.5 + (dt.hour + dt.minute / 60
                                       + dt.second / 3600
                                       + dt.microsecond / 3.6e9) / 24.0
    jd_int = int(jd)
    jd_frac = jd - jd_int
    return jd_int, jd_frac


def tle_to_dict(tle_data):
    """
    Parses a tle satellite dataset into a python dictionary.
    :param: tle_data
    :return: dictionary
    """
    tle_data_split = tle_data.strip().split("\n")
    tle_data_stripped = [line.strip() for line in tle_data_split]

    tle_list = []
    line_list = []


    for line in tle_data_stripped:
        line_list.append(line)
        if len(line_list) == 3:
            tle_list.append(line_list)
            line_list = []

    tle_dict = {}

    for sat in tle_list:
        name = sat[0]
        line_1 = sat[1]
        line_2 = sat[2]
        line_dict = {}

        #Line 1 values
        sat_number = int(line_1[2:7].strip())
        classification = line_1[7]
        Int_designation = line_1[9:17]
        epoch = datetime_to_jd(epoch_to_datetime(line_1[18:32]))
        first_derivative_of_mean_motion = float(line_1[33:43])
        second_derivative_of_mean_motion = tle_notation_to_float(line_1[44:52])
        bstar_drag_term = tle_notation_to_float(line_1[53:61])
        ephemeris_type = int(line_1[62])
        element_set_number = int(line_1[64:68])
        checksum_1 = int(line_1[68])

        #Line 2 values
        sat_number_2 = int(line_2[2:7].strip())
        inclination = float(line_2[8:16])
        raan = float(line_2[17:25])
        eccentricity = float("." + line_2[26:33].strip())
        perigee = float(line_2[34:42])
        mean_anomaly = float(line_2[43:51])
        mean_motion = float(line_2[52:63])
        revolution_number = int(line_2[63:68])
        checksum_2 = int(line_2[68])

        line_dict["Line1"] = {"satellite number": sat_number, "classification": classification, "international designator": Int_designation, "epoch": epoch,
                              "first derivative of mean motion": first_derivative_of_mean_motion, "second derivative of mean motion": second_derivative_of_mean_motion,
                              "BSTAR drag term": bstar_drag_term, "ephemeris type": ephemeris_type, "element set number": element_set_number, "checksum 1": checksum_1}

        line_dict["Line2"] = {"satellite number": sat_number_2, "inclination": inclination, "RAAN": raan, "eccentricity": eccentricity, "perigee": perigee,
                              "mean anomaly": mean_anomaly, "mean motion": mean_motion, "revolution number": revolution_number, "checksum 2": checksum_2}

        tle_dict[name] = line_dict

    return tle_dict


space_station_dict = tle_to_dict(get_tle_data("https://celestrak.org/NORAD/elements/gp.php?GROUP=stations&FORMAT=tle"))
space_station_dict

{'ISS (ZARYA)': {'Line1': {'satellite number': 25544,
   'classification': 'U',
   'international designator': '98067A  ',
   'epoch': (2460941, 0.7468678299337626),
   'first derivative of mean motion': 0.00027783,
   'second derivative of mean motion': 0.0,
   'BSTAR drag term': 0.00048333,
   'ephemeris type': 0,
   'element set number': 999,
   'checksum 1': 5},
  'Line2': {'satellite number': 25544,
   'inclination': 51.6338,
   'RAAN': 181.0318,
   'eccentricity': 0.0004269,
   'perigee': 9.4814,
   'mean anomaly': 350.6255,
   'mean motion': 15.50560509,
   'revolution number': 53037,
   'checksum 2': 7}},
 'CSS (TIANHE)': {'Line1': {'satellite number': 48274,
   'classification': 'U',
   'international designator': '21035A  ',
   'epoch': (2460942, 0.5161305600777268),
   'first derivative of mean motion': 0.00044487,
   'second derivative of mean motion': 0.0,
   'BSTAR drag term': 0.00053348,
   'ephemeris type': 0,
   'element set number': 999,
   'checksum 1': 2},
  'Line2'

### Import the SGP4 Propagotor library, propagate the ISS orbit, and get its state vector:

In [5]:
from sgp4.api import Satrec

def propagate(lines, time):
    ISS = Satrec.twoline2rv(lines[1], lines[2])
    jd_int, jd_frac = time
    error_code, position, velocity = ISS.sgp4(jd_int, jd_frac)
    return position, velocity

state_vector_params = propagate(tle_lines, space_station_dict["ISS (ZARYA)"]["Line1"]["epoch"])
state_vector_params


((-6790.004065243361, -122.29047406618665, 0.0009781515509994568),
 (0.09279529393946069, -4.7552327114024715, 6.0123250833190065))

### Propagate the ISS orbit 45 minutes into the future:

In [7]:
from sgp4.api import WGS72, Satrec
from datetime import datetime, timedelta, timezone

future_time = datetime.now(timezone.utc) + timedelta(minutes=45)

future_orbit = propagate(tle_lines, datetime_to_jd(future_time))

future_orbit

((1103.5240187150675, -4238.707879798008, 5183.270817540294),
 (7.539852572858038, 0.3077706614214961, -1.3460571915273944))